- Es importante tener en cuenta que los precios pasados no garantizan resultados futuros, por eso podemos utilizar:

Técnicas de aprendizaje automático para predecir los precios de las acciones en el futuro y se explorará si hay algún patrón en el comportamiento de las acciones según el día de la semana.

In [2]:
#Librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
import datetime as dt

In [3]:
# Obtener datos de Yahoo Finance
tickers_sec= ["XLY", "XLK", "XLV", "XLI"]
data_pred = yf.download(tickers_sec, start="2000-01-01", end="2023-02-23")

[*********************100%***********************]  4 of 4 completed


In [4]:
#Filtro por precio ajustado
data=data_pred["Adj Close"]

In [5]:
# Vistazo
data.head()

,XLI,XLK,XLV,XLY
Date,,,,
2000-01-03,18.979509,42.209297,21.571779,22.924379
2000-01-04,18.455160,40.067913,21.082268,22.234743
2000-01-05,18.372908,39.473091,20.893137,21.961264
2000-01-06,18.619659,38.164455,20.959890,22.210960
2000-01-07,19.329082,38.830666,21.215769,23.257296


In [6]:
# Crear un objeto de regresión lineal
reg = LinearRegression()

In [7]:
# Crear un diccionario vacío para almacenar los DataFrames de predicciones para cada ticker
predictions_dict = {}

# Iterar sobre cada ticker del sector
for ticker in tickers_sec:
    # Seleccionar el ticker actual
    ticker_data = data[ticker]
    
    # Definir la variable de entrada X (fechas)
    X = np.array(ticker_data.index.astype(int) // 10**9).reshape(-1,1)
    
    # Definir la variable de salida Y (precios ajustados)
    Y = np.array(ticker_data.values)
    
    # Entrenar un modelo de regresión lineal
    reg = LinearRegression().fit(X, Y)
    
    # Definir las fechas futuras para las que se quieren hacer predicciones
    future_dates = pd.date_range(start="2023-02-23", periods=365, freq="D")
    future_date_values = np.array(future_dates.astype(int) // 10**9)
    
    # Hacer predicciones de precios para las fechas futuras utilizando el modelo de regresión lineal
    price_predictions = reg.predict(future_date_values.reshape(-1,1))
    
    # Crear un DataFrame que contenga las fechas futuras y las predicciones de precios
    predictions_df = pd.DataFrame({
        "Date": future_dates,
        "Price Prediction": price_predictions.flatten(),
        "Ticker": ticker
    })
    
    # Agregar el DataFrame de predicciones para el ticker actual al diccionario
    predictions_dict[ticker] = predictions_df
    
# Combinar todos los DataFrames en uno solo
predictions_combined = pd.concat(predictions_dict.values(), ignore_index=True)

# Imprimir el DataFrame combinado
print(predictions_combined)

           Date  Price Prediction Ticker
0    2023-02-23        131.558727    XLY
1    2023-02-24        131.575783    XLY
2    2023-02-25        131.592839    XLY
3    2023-02-26        131.609896    XLY
4    2023-02-27        131.626952    XLY
...         ...               ...    ...
1455 2024-02-18         84.378349    XLI
1456 2024-02-19         84.387836    XLI
1457 2024-02-20         84.397323    XLI
1458 2024-02-21         84.406810    XLI
1459 2024-02-22         84.416297    XLI

[1460 rows x 3 columns]


C:\Users\Jhonatan\AppData\Local\Temp\ipykernel_15140\612750010.py:10: FutureWarning: The behavior of .astype from datetime64[ns] to int32 is deprecated. In a future version, this astype will return exactly the specified dtype instead of int64, and will raise if that conversion overflows.
  X = np.array(ticker_data.index.astype(int) // 10**9).reshape(-1,1)
C:\Users\Jhonatan\AppData\Local\Temp\ipykernel_15140\612750010.py:20: FutureWarning: The behavior of .astype from datetime64[ns] to int32 is deprecated. In a future version, this astype will return exactly the specified dtype instead of int64, and will raise if that conversion overflows.
  future_date_values = np.array(future_dates.astype(int) // 10**9)
C:\Users\Jhonatan\AppData\Local\Temp\ipykernel_15140\612750010.py:10: FutureWarning: The behavior of .astype from datetime64[ns] to int32 is deprecated. In a future version, this astype will return exactly the specified dtype instead of int64, and will raise if that conversion overflow

In [8]:
# Nombres unicos de la columna Ticker
predictions_combined["Ticker"].unique()

array(['XLY', 'XLK', 'XLV', 'XLI'], dtype=object)

In [9]:
# Vistazo
predictions_combined

,Date,Price Prediction,Ticker
0,2023-02-23,131.558727,XLY
1,2023-02-24,131.575783,XLY
2,2023-02-25,131.592839,XLY
3,2023-02-26,131.609896,XLY
4,2023-02-27,131.626952,XLY
...,...,...,...
1455,2024-02-18,84.378349,XLI
1456,2024-02-19,84.387836,XLI
1457,2024-02-20,84.397323,XLI
1458,2024-02-21,84.406810,XLI


In [10]:
# Guardo datos para la presentacion
#predictions_combined.to_excel("/content/drive/MyDrive/Giselle PI/Proyecto02/predi.xlsx", index=False)

En el DF predictions_combined se encuentran las predicciones juntas, asi que las separamos:

In [11]:
xly_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLY"]
xlk_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLK"]
xlv_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLV"]
xli_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLI"]

In [12]:
#xly_df.to_excel("/content/drive/MyDrive/Giselle PI/Proyecto02/XLY.xlsx", index=False)
#xlk_df.to_excel("/content/drive/MyDrive/Giselle PI/Proyecto02/XLK.xlsx", index=False)
#xlv_df.to_excel("/content/drive/MyDrive/Giselle PI/Proyecto02/XLV.xlsx", index=False)
#xli_df.to_excel("/content/drive/MyDrive/Giselle PI/Proyecto02/XLI.xlsx", index=False)

Vistazo de cada una:

In [13]:
xly_df

,Date,Price Prediction,Ticker
0,2023-02-23,131.558727,XLY
1,2023-02-24,131.575783,XLY
2,2023-02-25,131.592839,XLY
3,2023-02-26,131.609896,XLY
4,2023-02-27,131.626952,XLY
...,...,...,...
360,2024-02-18,137.698946,XLY
361,2024-02-19,137.716002,XLY
362,2024-02-20,137.733058,XLY
363,2024-02-21,137.750115,XLY


In [14]:
xlk_df

,Date,Price Prediction,Ticker
365,2023-02-23,95.211110,XLK
366,2023-02-24,95.223491,XLK
367,2023-02-25,95.235872,XLK
368,2023-02-26,95.248253,XLK
369,2023-02-27,95.260634,XLK
...,...,...,...
725,2024-02-18,99.668258,XLK
726,2024-02-19,99.680639,XLK
727,2024-02-20,99.693020,XLK
728,2024-02-21,99.705401,XLK


In [15]:
xlv_df

,Date,Price Prediction,Ticker
730,2023-02-23,102.814522,XLV
731,2023-02-24,102.827243,XLV
732,2023-02-25,102.839963,XLV
733,2023-02-26,102.852683,XLV
734,2023-02-27,102.865403,XLV
...,...,...,...
1090,2024-02-18,107.393813,XLV
1091,2024-02-19,107.406533,XLV
1092,2024-02-20,107.419254,XLV
1093,2024-02-21,107.431974,XLV


In [16]:
xli_df

,Date,Price Prediction,Ticker
1095,2023-02-23,80.963004,XLI
1096,2023-02-24,80.972491,XLI
1097,2023-02-25,80.981978,XLI
1098,2023-02-26,80.991466,XLI
1099,2023-02-27,81.000953,XLI
...,...,...,...
1455,2024-02-18,84.378349,XLI
1456,2024-02-19,84.387836,XLI
1457,2024-02-20,84.397323,XLI
1458,2024-02-21,84.406810,XLI
